In [13]:
import os
import sys
from anvil.terra.api import FAPI
from anvil.terra.api import get_projects, get_entities
from anvil.terra import api

sys.path.insert(0, os.path.abspath('../../src/python'))
from trackview_utils import generate_signed_url_v4, encode_uri_component

In [14]:
# Terra workspace in the form of 'namespace/name'

workspace_project = "Calico_SETDB1"
workspace_namespace = "SETDB1-point-mutants"
workspace_table = "chip_march_2024_merged"

json_credentials = "" # Path to the JSON credentials file for a service account authorized in Terra

table = FAPI.get_entities(workspace_project, workspace_namespace, workspace_table).json()
print(len(table))

21


In [17]:
ucsc_custom_text = []
ucsc_custom_text.append( "browser position chr19:11500000-12000000")

track_type = "bigWig"
track_name = ""
track_description = ""
track_windowing_function = "mean"
track_visibility = "full"
track_max_height_pixels = 70
track_auto_scale = "off"
track_view_limits = "0:70"
track_color = "0,0,0"
track_big_data_url = ""

#TODO: choose the color on the track based on the epitope or track name

# IMPORTANT: Change the name of the keys according to your Terra table
for attribute in table:
    row = attribute["attributes"]
    track_name = attribute['name']
    track_description = attribute['name']
    bucket_name, file_name = row['igvtools_count_bw'][5:].split("/", 1)
    track_big_data_url = generate_signed_url_v4(bucket_name, file_name, json_credentials)
    if 'distinct_fragments' in row:
        distinct_fragments = str(row['distinct_fragments'])
        track_view_limits = f"0:{distinct_fragments[:2]}"
    ucsc_custom_text.append(F"track type={track_type} name=\"{track_name}\" description=\"{track_description}\" windowingFunction={track_windowing_function} visibility={track_visibility} maxHeightPixels={track_max_height_pixels} autoScale={track_auto_scale} viewLimits={track_view_limits} color={track_color} bigDataUrl={track_big_data_url}")

# Open the file in write mode ('w')
with open('custom_text.txt', 'w') as file:
    # Write the joined string to the file
    file.write('\n'.join(ucsc_custom_text))

print(len(ucsc_custom_text))

22


In [18]:
# the custom_text.txt file needs to be uploaded to a public bucket
sanitized_custom_text_link = encode_uri_component("https://storage.googleapis.com/path/custom_text.txt")
# session.txt is in this repo in /data/ucsc/session.txt
sanitized_session_link = encode_uri_component("https://storage.googleapis.com/path/session.txt")
ucsc_final_url = f"https://genome.ucsc.edu/cgi-bin/hgTracks?db=mm10&hgt.customText={sanitized_custom_text_link}&hgS_doLoadUrl=submit&hgS_loadUrlName={sanitized_session_link}"
print(ucsc_final_url)

https://genome.ucsc.edu/cgi-bin/hgTracks?db=mm10&hgt.customText=https%3A%2F%2Fstorage.googleapis.com%2Fbroad-bernstein-ucsc-trackhubs%2Fucsc%2Fcustom_text2.txt&hgS_doLoadUrl=submit&hgS_loadUrlName=https%3A%2F%2Fstorage.googleapis.com%2Fbroad-bernstein-ucsc-trackhubs%2Fucsc%2Fsession.txt
